# Análisis de serie de tiempo con ARIMA

**Autor:** Roberto Muñoz <br />
**E-mail:** <rmunoz@metricarts.com> <br />
**Github:** <https://github.com/rpmunoz> <br />

ARIMA o Modelo autorregresivo integrado de media móvil es un modelo estadístico que utiliza variaciones y regresiones de datos estadísticos con el fin de encontrar patrones para una predicción hacia el futuro. Se trata de un modelo dinámico de series temporales, es decir, las estimaciones futuras vienen explicadas por los datos del pasado y no por variables independientes.

Usaremos la librería [Statsmodels](https://www.statsmodels.org) para hacer análisis estadístico.

## Importar librerías

In [ ]:
# Import libraries
import warnings
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Defaults
plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams.update({'font.size': 12})
plt.style.use('ggplot')

pd.set_option('display.max_columns', 500)

Cargamos la BBDD de ventas mensuales de lavadoras frontales

In [ ]:
# Load the data
data_df = pd.read_excel('data/BBDDMensualLavFront.xlsx')
data_df.head()

In [ ]:
data_df.columns

In [ ]:
data_df['datetime'] = pd.to_datetime(dict(year=data_df['Año'], month=data_df['Mes'], day=1.))
data_df.set_index(['datetime'], inplace=True)
data_df = data_df.sort_index()
data_df.head()

In [ ]:
print(data_df.index.min())
print(data_df.index.max())

## Agrupamos por mes del año

In [ ]:
columns_mask = ['Unidades']
columns_groupby = ['datetime']

In [ ]:
unidades_df = data_df[columns_mask].groupby('datetime').sum()
unidades_df.head()

In [ ]:
unidades_df.plot()
plt.ylabel('Monto')
plt.xlabel('Fecha')
plt.show()

## Hacemos análisis de ARIMA

In [ ]:
# Define the d and q parameters to take any value between 0 and 1
q = d = range(0, 4)
# Define the p parameters to take any value between 0 and 3
p = range(0, 4)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

print(pdq)

In [ ]:
# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

Seleccionamos un subconjunto de la serie de datos como datos de entrenamiento, digamos los primeros 11 años. Nuestro objetivo es predecir el último año de la serie en función de esta información.

In [ ]:
train_data = unidades_df['2017-01-01':'2018-12-01']
test_data = unidades_df['2018-01-01':'2018-12-01']

In [ ]:
warnings.filterwarnings("ignore") # specify to ignore warning messages

AIC = []
SARIMAX_model = []
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(train_data,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('SARIMAX{}x{} - AIC:{}'.format(param, param_seasonal, results.aic), end='\r')
            AIC.append(results.aic)
            SARIMAX_model.append([param, param_seasonal])
        except:
            continue

In [ ]:
print('El valor de AIC más pequeño es {} para el modelo SARIMAX{}x{}'.format(min(AIC), SARIMAX_model[AIC.index(min(AIC))][0],SARIMAX_model[AIC.index(min(AIC))][1]))


In [ ]:
# Ajustemos el modelo
mod = sm.tsa.statespace.SARIMAX(train_data,
                                order=SARIMAX_model[AIC.index(min(AIC))][0],
                                seasonal_order=SARIMAX_model[AIC.index(min(AIC))][1],
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()

In [ ]:
results.summary()

In [ ]:
results.plot_diagnostics(figsize=(20, 14))
plt.show()

### Hacemos las predicciones

In [ ]:
pred0 = results.get_prediction(start='2018-01-01', dynamic=False)
pred0_ci = pred0.conf_int()

pred1 = results.get_prediction(start='2018-01-01', dynamic=True)
pred1_ci = pred1.conf_int()

pred2 = results.get_forecast('2019-12-01')
pred2_ci = pred2.conf_int()

In [ ]:
ax = unidades_df.plot(figsize=(20, 16))
pred0.predicted_mean.plot(ax=ax, label='1-step-ahead Forecast (get_predictions, dynamic=False)')
pred1.predicted_mean.plot(ax=ax, label='Dynamic Forecast (get_predictions, dynamic=True)')
pred2.predicted_mean.plot(ax=ax, label='Dynamic Forecast (get_forecast)')
ax.fill_between(pred2_ci.index, pred2_ci.iloc[:, 0], pred2_ci.iloc[:, 1], color='k', alpha=.1)
plt.ylabel('Numero de unidades vendidas')
plt.xlabel('Fecha')
plt.legend()
plt.show()

Calculamos el error de la predicción usando los datos 2018

In [ ]:
prediction = pred0.predicted_mean['2018-01-01':'2018-12-01'].values
test_data = unidades_df['2018-01-01':'2018-12-01']

# flatten nested list
truth = list(itertools.chain.from_iterable(test_data.values))

# Mean Absolute Percentage Error
MAPE = np.mean(np.abs((truth - prediction) / truth)) * 100

print('El MAE porcentula para el año 2018 es {:.2f}%'.format(MAPE))

Obtenemos los valores de la predicción a futuro

In [ ]:
print(pred2.predicted_mean['2019-01-01':'2019-12-01'])

## Agrupamos usando la columna Segmento

In [ ]:
columns_mask = ['Segmento','Unidades']
columns_groupby = ['Segmento']

groups_df = data_df[columns_mask].groupby(columns_groupby)

for name, group_df in groups_df:
    print("Group name: ", name)
    print(group_df.head())
    print()

In [ ]:
unidades_df = groups_df.get_group('High').groupby('datetime').sum()
unidades_df.head()

In [ ]:
unidades_df.plot()
plt.ylabel('Numero de unidades vendidas')
plt.xlabel('Fecha')
plt.show()

Mirando la figura, el modelo parece hacer un muy buen trabajo al modelar la serie temporal. Las líneas azules y púrpuras son, como se esperaba, muy cercanas a la verdad del fondo rojo. Lo que es más interesante es la línea gris, la predincción fuera de la muestra. Para una serie temporal tan simple, el modelo ARIMA puede pronosticar los valores de 1960 con precisión.

Para cuantificar la precisión de la predicción para 1960, podemos calcular métricas como Error absoluto medio (MAE), Error cuadrado medio (MSE) o Error cuadrado medio raíz (RMSE).

Estas son todas métricas absolutas, por lo tanto, dependen de la escala. Si bien son útiles y se usan ampliamente para comparar diferentes métodos en un mismo conjunto de datos, aquí sería más útil expresar el error en relación con la magnitud de la serie de tiempo que estamos tratando de predecir. Una medida de error porcentual de uso común es el error porcentual absoluto medio (MAPE). Tiene algunos inconvenientes en casos especiales (en particular, puede conducir a la división por cero) y se han propuesto medidas mejoradas, ver p. Ej. https://www.otexts.org/fpp/2/5 para una buena visión general. Sin embargo, para este ejemplo nos quedaremos con MAPE.

In [ ]:
prediction = pred2.predicted_mean['1960-01-01':'1960-12-01'].values
# flatten nested list
truth = list(itertools.chain.from_iterable(test_data.values))
# Mean Absolute Percentage Error
MAPE = np.mean(np.abs((truth - prediction) / truth)) * 100

print('The Mean Absolute Percentage Error for the forecast of year 1960 is {:.2f}%'.format(MAPE))